---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Read the text file as csv into a dataframe.
    # Using newline as the separator
    utown_df = pd.read_csv("university_towns.txt",sep='\n',header=None, engine='python')
    utown_df.columns = ['names']
    
    # Create a new column 'States' by:
    #    splitting the entry by '[edit]', if present, else assigning it NaN value
    #    picking up the first element of the split which would be the state name
    #    striping of any whitespace
    #    forward filling any NaN value with the state name
    utown_df['State'] = utown_df['names'].apply(lambda x: x.split('[edit]')[0].strip()
                                               if x.count('[edit]')>0 else np.NaN).fillna(method='ffill')
    
    # Create a new column 'RegionName' by:
    #    splitting the entry by '(', if '[edit]' not present, else assigning it NaN value
    #    picking up the first element of the split which would be the Region name
    #    striping of any whitespace
    utown_df['RegionName'] = utown_df['names'].apply(lambda x: x.split('(')[0].strip()
                                               if x.count('[edit]')==0 else np.NaN)
    
    # drop any rows with NaN
    # drop 'names', the original column
    # reset the index to start from 0 while dropping the original index
    utown_df = utown_df.dropna().drop('names', axis=1).reset_index(drop=True)
    
    return utown_df

In [5]:
print(get_list_of_university_towns())

             State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas  

In [6]:
def read_gdp_data():
    
    # Read the gdp excel file
    #    take no header
    #    parse only the quarters and gdp (chained value in 2009 dollars) columns
    #    not make any column as index
    #    skip the first 8 rows (mostly blank rows and headers)
    #    convert any decimal value to float (although it does the conversion even without specifying, in this case)
    gdp_df = pd.read_excel('gdplev.xls', header=None, parse_cols=[4,6], index_col=None, skiprows= 8, convert_float=True)
    
    # Rename the columns
    gdp_df.columns = ['Quarter', 'GDP']
    
    #Find the index of 2000q1, the starting quarter of interest
    start_index = gdp_df[gdp_df['Quarter']=='2000q1'].index[0]
    
    # Taking the subset of the data from 2000q1
    gdp_df = gdp_df[start_index:].reset_index(drop=True)
    
    return gdp_df

def label_gdp():
    
    # Get the GDP data frame and create a new column 'change' with default = 0 (1 - GDP increase, 0 - GDP decline )
    gdp_df = read_gdp_data()
    gdp_df['change'] = '0'
    
    # Label the 'change' column with 1 or 0 depending on if the GDP increased compared to previous quarter
    for i in range(1, gdp_df.shape[0]):
        if gdp_df['GDP'][i] > gdp_df['GDP'][i-1]:
            gdp_df['change'][i] = '1'
        
    return gdp_df

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = label_gdp()
    
    # Compare consecutive quarters to check GDP decline in 2 quarters in a row
    # Recession start will be the quarter where the decline began
    recession_start = ''
    for i in range(1, gdp_df.shape[0]):
        if (gdp_df['change'][i] == '0') and (gdp_df['change'][i-1] == '0'):
            recession_start = gdp_df['Quarter'].loc[i-1]
            break
    
    return recession_start

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = label_gdp()
    recession_start = get_recession_start()
    
    # Find the index of receession start date
    recession_start_index = gdp_df[gdp_df['Quarter'] == recession_start].index[0]
    
    recession_end = ''
    
    # Start a loop from recession start quarter +3
    # 0,1 are decline - from recession start; 2, 3 will be compared in the first run) 
    # If two consecutive quarters (after recession start) show GDP increase - the second quarter is recession end
    for i in range(recession_start_index+3, gdp_df.shape[0]):
        if (gdp_df['change'][i] == '1') and (gdp_df['change'][i-1] == '1'):
            recession_end = gdp_df['Quarter'].loc[i]
            break
       
    return recession_end

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # Get the GDP dataframe (Set Quarters as index) and recession start and end quarters
    gdp_df = read_gdp_data().set_index('Quarter',drop =True)
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    # Find the index of the minimum GDP between recession start and end quarters
    recession_bottom = gdp_df.loc[recession_start:recession_end].idxmin()[0]
    
    return recession_bottom

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # Read the Zillow Housing Price file into a data frame
    raw_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    #Declare the States dictionary
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    # Change the State column to have full state names, using the states dictionary
    raw_df['State'] = raw_df['State'].apply(lambda x: states[x])
    
    # Set the index to [State,Region Name]
    raw_df = raw_df.set_index(['State', 'RegionName'])
    
    # Create a new dataframe that holds only Monthly median prices and convert it to datetime
    time_cols_df = raw_df[raw_df.columns[4:]].rename(columns=pd.to_datetime)
    cols = time_cols_df.columns
    
    # Select Columns from 2000 Quarter 1
    # Resample it by mean of quarters
    # rename the Columns to format 2000q1
    time_cols_df = time_cols_df[cols[cols >= '2000-01-01']] \
                   .resample('Q', axis = 1).mean() \
                   .rename(columns = lambda x: '{}q{}'.format(x.year, x.quarter)) 
    
    return time_cols_df

In [10]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Get the GDP dataframe (Set Quarters as index) and recession start, end, bottom quarters
    gdp_df = read_gdp_data()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = get_recession_bottom()
    utown_df = get_list_of_university_towns()
    housing_df = convert_housing_data_to_quarters()
    
    recession_start_index = gdp_df[gdp_df['Quarter'] == recession_start].index[0]
    quarter_before_recession = gdp_df['Quarter'].loc[recession_start_index-1]
    
    housing_df['PriceRatio'] = housing_df[quarter_before_recession].div(housing_df[recession_bottom])
    
    utown_tup = [tuple(x) for x in utown_df.to_records(index = False)]
    
    housing_utown = housing_df.loc[utown_tup]
    housing = housing_df.loc[~housing_df.index.isin(utown_tup)]
    
    utown_mean = housing_utown['PriceRatio'].mean()
    rest_mean = housing['PriceRatio'].mean()
    
    if utown_mean < rest_mean:
        better = 'university town'
    else:
        better = 'non-university town'
    
    ttest = stats.ttest_ind(housing_utown['PriceRatio'], housing['PriceRatio'], nan_policy='omit')
    
    if ttest[1] < 0.01:
        different = True
    else:
        different = False
        
    return (different, ttest[1], better)

In [13]:
print(run_ttest())

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0027240637047531249, 'university town')
